# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        fail = True
        while fail:
            bgnDex   = randrange(0,Nchunks) * chunkSize
            moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
            sendDex  = randrange(0,Ntracks-1)
            try:
                response = spot.playlist_reorder_items(
                    playlist_ID, 
                    bgnDex, 
                    sendDex, 
                    range_length = chunkSize
                )
                if verbose:
                    print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
                fail = False
            except spotipy.SpotifyException as ex:
                if verbose:
                    print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
    'zk_Over' : "1diwH003mkfg3cLejTuKTN",
}

complete_reorder_entire_playlist( playlist['zk_Over'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 700-800 --> 944:
	{'snapshot_id': 'AAAF0VjOMd8ID2jmqEfE8x+vMwJl4JjR'}
Iteration 2, 0-100 --> 1001:
	{'snapshot_id': 'AAAF0u1s9HnYxJNzwJguU2vqEi7HYqJ3'}
Iteration 3, 700-800 --> 975:
	{'snapshot_id': 'AAAF06ccMv8S2K8Md0xB9X1EdKHBekAa'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 4, 100-200 --> 602:
	{'snapshot_id': 'AAAF1EufoeOwKatV2KPwAoHMt4FykEgd'}
ERROR, 1000-1020 --> 179:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 5, 900-1000 --> 983:
	{'snapshot_id': 'AAAF1U/D6GszSXajrmFSDD8NmQjuunna'}
Iteration 6, 200-300 --> 138:
	{'snapshot_id': 'AAAF1hdcYavltyo0Op1OdnwZKK45ycW9'}
Iteration 7, 800-900 --> 271:
	{'snapshot_id': 'AAAF13mgc6f/a2KSead/5tqvlf7pmP1c'}
Iteration 8, 200-300 --> 609:
	{'snapshot_id': 'AAAF2Nc4jeIxXfJnHzBISkAQi/13e6ke'}
Iteration 9, 800-900 --> 127:
	{'snapshot_id': 'AAAF2VjC7fTPOLoGLLxNAJ+7o+vq2DPv'}
Iteration 10, 800-900 --> 112:
	{'snapshot_id': 'AAAF2rNF1v1iWcvFB3o0Aa8xiogWYGhl'}

Phase 2: Chunks of 10

Iteration 1, 570-580 --> 335:
	{'snapshot_id': 'AAAF20khkLcQ8h4kWEzZCBO8pB+of9ms'}
Iteration 2, 500-510 --> 343:
	{'snapshot_id': 'AAAF3JW13/j97i5HpnMbHTaK+04S/Jlb'}
Iteration 3, 40-50 --> 324:
	{'snapsho

HTTP Error for PUT to https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 26, 780-790 --> 851:
	{'snapshot_id': 'AAAF9JbkuNjiHPhu9hYg3ipxVBWZJnBz'}
ERROR, 1020-1020 --> 625:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/1diwH003mkfg3cLejTuKTN/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 27, 440-450 --> 840:
	{'snapshot_id': 'AAAF9c4q8AFbrZWgGDVuXmmiPl4IJZgS'}
Iteration 28, 360-370 --> 850:
	{'snapshot_id': 'AAAF9m91YsACo8zYKZgXXzVjY8uT3xZY'}
Iteration 29, 180-190 --> 624:
	{'snapshot_id': 'AAAF93S0X62EWirPKdLh1BqqQimFhHVW'}
Iteration 30, 250-260 --> 760:
	{'snapshot_id': 'AAAF+LPaCRwTLV2tL7l8yqa/030m7vBd'}
Iteration 31, 650-660 --> 862:
	{'snapshot_id': 'AAAF+ShuSkQUSE5qZ8jZhnBXwDhSUaZ9'}
Iteration 32, 340-350 --> 164:
	{'snapshot_id': 'AAAF+nC5btiRhkAPib8Zg2/XVW75hbcz'}
Iteration 33, 80-90 --> 655:
	{'snapshot_id': 'AAAF+6r7mQts9lp88ketzt4gXyAbd+LR'}
Iteration 34, 880-890 --> 412:
	{'snapshot_id': 'AAAF/AkudZW24llyLnBNJHpQJZq/DUul'}
Iteration 35, 240-250 --> 949:
	{'snapshot_id': 'AAAF/RPL